# Análise Completa - Case Ifood: Teste A/B Estratégia de Cupons

Notebook único para orquestrar as tarefas de execução de *setup*, **ETL** e análise dos dados, integrando os diferentes módulos do repositório de origem:

- Clona/atualiza o repositório do projeto, com as dependências, no Colab
- Instala dependências e faz o **download** dos dados brutos
- Sobe Spark e executa o **ETL** (orders/consumers/restaurants + mapa A/B)
- Mantém `orders_silver` e `users_silver` em memória
- Realiza a análise exploratória dos dados


## Configuração do Ambiente e Preparação dos Dados

### Configuração de Ambiente e Download de Dados Brutos

In [ ]:
import os, sys, subprocess
from pathlib import Path

GITHUB_USER = "silvaniacorreia"
REPO_NAME   = "ifood-case-cupons"
REPO_URL    = f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git"

def run(cmd):
    print(">", " ".join(cmd))
    subprocess.check_call(cmd)

# 1) clonar/atualizar repositório
ROOT = Path("/content")
PROJECT_DIR = ROOT / REPO_NAME
if not PROJECT_DIR.exists():
    run(["git", "clone", REPO_URL, str(PROJECT_DIR)])
else:
    os.chdir(PROJECT_DIR)
    run(["git", "fetch", "--all"])
    run(["git", "checkout", "main"])
    run(["git", "pull", "--rebase", "origin", "main"])
os.chdir(PROJECT_DIR)

# 2) deps + download programático
run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "--no-cache-dir"])
run([sys.executable, "scripts/download_data.py"])

# 3) sys.path
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))
print("✔️ Bootstrap concluído. Projeto:", PROJECT_DIR)


### Iniciando o Spark

In [ ]:
from src.utils import load_settings, get_spark

s = load_settings()
spark = get_spark(
    app_name=s.runtime.spark.app_name,
    shuffle_partitions=s.runtime.spark.shuffle_partitions,
    extra_conf=getattr(s.runtime.spark, "conf", {}) 
)
print("✔️ Spark ativo - versão:", spark.version)

# checagem rápida
spark.range(5).show()


### Análises Pré-Flight

In [ ]:
## Checagens dados brutos
from src.checks import preflight
from pprint import pprint

rep = preflight(s.data.raw_dir, strict=False)
print("Pré-flight (resumo):")
pprint({
    "raw_dir": rep["raw_dir"],
    "orders_format_guess": rep["orders_format_guess"],
    "files": {k: {kk: vv for kk, vv in v.items() if kk in ("exists","size_bytes","gzip_ok","tar_ok")} for k, v in rep["files"].items()},
    "ab_csv_candidates": rep["ab_csv_candidates"][:3],
})


### ETL (Extração, Transformação e Carga)

In [ ]:
from src import etl, checks
from pyspark.sql import functions as F
import os

def _get_exp_window(s):
    """
    Lê a janela do experimento a partir das configurações. Caso não exista, utiliza inferência automática.

    Parâmetros:
        s: Objeto de configurações carregado.

    Retorna:
        Tuple[str, str, bool]: Data de início, data de fim e flag de inferência automática.
    """
    win = getattr(s.analysis, "experiment_window", None)
    if isinstance(win, dict):
        start = win.get("start")
        end   = win.get("end")
    else:
        start = None
        end   = None
    auto = bool(getattr(s.analysis, "auto_infer_window", True))
    return start, end, auto

start, end, auto = _get_exp_window(s)

# 1) Leitura dos dados brutos
orders, consumers, restaurants, abmap = etl.load_raw(spark, s.data.raw_dir)
checks.profile_loaded(orders, consumers, restaurants, abmap, n=5)

# 2) Limpeza e conformidade dos dados
# Inclui normalização de timezone e aplicação de janela experimental
# Utiliza quantis para robustez contra outliers
df = etl.clean_and_conform(
    orders, consumers, restaurants, abmap,
    business_tz=getattr(s.analysis, "business_tz", "America/Sao_Paulo"),
    treat_is_target_null_as_control=getattr(s.analysis, "treat_is_target_null_as_control", False),
    experiment_start=start,
    experiment_end=end,
    auto_infer_window=auto,
    use_quantile_window=True,     # janela robusta a outliers
    verbose=True
)

# 3) Ajustes finais e agregações para análise
orders_silver = etl.build_orders_silver(df)
orders_silver = etl.enrich_orders_for_analysis(orders_silver)
users_silver  = etl.build_user_aggregates(orders_silver)

# 4) Cálculo de recência com base no último timestamp observado
ref_ts = orders_silver.agg(F.max("event_ts_utc")).first()[0]
users_silver = users_silver.withColumn("recency", F.datediff(F.lit(ref_ts), F.col("last_order")))

# 5) Salvar resultados em formato Parquet (opcional)
SAVE_PARQUET = False
if SAVE_PARQUET:
    (
        orders_silver
        .write
        .mode("overwrite")
        .partitionBy("event_date_brt")
        .parquet(f"{s.data.processed_dir}/orders_silver.parquet")
    )
    users_silver.write.mode("overwrite").parquet(f"{s.data.processed_dir}/users_silver.parquet")

# 6) Contagem de linhas para validação
print("orders_silver:", orders_silver.count(), "linhas")
print("users_silver :", users_silver.count(), "linhas")

# 7) Exibição de amostras para validação
try:
    from IPython.display import display
    display(orders_silver.limit(5).toPandas())
    display(users_silver.limit(5).toPandas())
except Exception as e:
    print("Aviso: toPandas falhou, mostrando via Spark .show()")
    orders_silver.show(5, truncate=False)
    users_silver.show(5, truncate=False)

### Checagem dos Dados

In [ ]:
# --- Checks essenciais ---
from pyspark.sql import functions as F

def check_post_etl(
    orders_silver,
    users_silver,
    *,
    light: bool = True,
    key_cols: list[str] | None = None,
    sample_frac: float = 0.001,  
    preview_rows: int = 5,
    use_pandas_preview: bool = False,  
):
    """
    light=True: nulos só em colunas-chave, preview por sample e show().
    light=False: nulos em todas as colunas (processamento lento).
    """
    if key_cols is None:
        key_cols = [
            # chaves/temporal/valor
            "order_id", "customer_id", "merchant_id",
            "event_ts_utc", "order_total_amount",
            # A/B e dimensões principais
            "is_target", "price_range", "language", "active",
            # atributos do restaurante usados na análise
            "delivery_time_imputed", "minimum_order_value_imputed"
        ]
    key_cols = [c for c in key_cols if c in orders_silver.columns]

    print("Faixa de datas (UTC) em orders_silver:")
    orders_silver.agg(
        F.min("event_ts_utc").alias("min_utc"),
        F.max("event_ts_utc").alias("max_utc")
    ).show(truncate=False)

    print("Split A/B (users):")
    users_silver.groupBy("is_target").count().orderBy("is_target").show()

    # --- Nulos em orders_silver ---
    def nulls_by_col(df, cols):
        exprs = [F.sum(F.col(c).isNull().cast("int")).alias(c) for c in cols]
        return df.select(exprs)

    if light:
        print(f"Nulos (colunas-chave): {key_cols}")
        nulls_by_col(orders_silver, key_cols).show(truncate=False)
    else:
        print("Nulos (todas as colunas) — operação pesada:")
        nulls_by_col(orders_silver, orders_silver.columns).show(truncate=False)

    # --- Previews rápidos ---
    print("\nPreview orders_silver (sample leve):")
    orders_preview_cols = [c for c in ["price_range","order_id","customer_id","merchant_id",
                                       "event_ts_utc","order_total_amount","origin_platform",
                                       "is_target","language","active"] if c in orders_silver.columns]
    preview_df = orders_silver.sample(False, sample_frac, seed=42).select(*orders_preview_cols)
    if preview_df.rdd.isEmpty():
        preview_df = orders_silver.select(*orders_preview_cols).limit(preview_rows)
    preview_df.show(preview_rows, truncate=False)

    print("\nPreview users_silver (primeiras linhas):")
    users_preview_cols = [c for c in ["customer_id","last_order","frequency","monetary","is_target","recency"]
                          if c in users_silver.columns]
    users_silver.select(*users_preview_cols).show(preview_rows, truncate=False)

    if use_pandas_preview:
        try:
            from IPython.display import display
            display(orders_silver.limit(preview_rows).toPandas())
            display(users_silver.limit(preview_rows).toPandas())
        except Exception:
            pass

# --- Executar em modo leve ---
check_post_etl(orders_silver, users_silver, light=True)


## A/B de cupons

### Importações e Configurações

In [ ]:
from pyspark.sql import functions as F
from src.analysis_ab import (
    compute_ab_summary,
    collect_user_level_for_tests,
    run_ab_tests,
    financial_viability
)
from src.utils import load_settings
settings = load_settings("config/settings.yaml")


### Métricas por grupo

In [ ]:
ab_summary_spark = compute_ab_summary(users_silver)
ab_summary_spark.show(truncate=False)

### Testes de significância

In [ ]:
users_pdf = collect_user_level_for_tests(users_silver)
tests = run_ab_tests(users_pdf)
tests


### Viabilidade financeira

take_rate = settings.finance.take_rate
coupon_cost = settings.finance.coupon_cost_default

# Cenário 1 (padrão): redemption_rate = conversão do tratamento (aproximação superior)
finance_default = financial_viability(
    users_pdf,
    take_rate=take_rate,
    coupon_cost=coupon_cost,
    redemption_rate=None
)

# Cenário 2 (conservador): defina uma taxa de resgate explícita (ex.: 30%)
finance_conservador = financial_viability(
    users_pdf,
    take_rate=take_rate,
    coupon_cost=coupon_cost,
    redemption_rate=0.30
)

finance_default, finance_conservador
